### Purpose: entry point of the app.

    - Imports engine & SessionLocal from db/config.py.
    - Imports models (so SQLAlchemy knows about them).
    - Calls Base.metadata.create_all() to create tables.
    - Runs a simple CRUD test.

In [1]:
## Imports

from sqlalchemy import or_, and_, select  # Importing SQLAlchemy functions for complex queries.
from db.config import engine, SessionLocal  # Brings in the engine (connection to the database) and SessionLocal (our session factory for talking to DB).
from db.base import Base  # Imports the Base class. This holds all models’ metadata (like table definitions).
from models.user import User
from models.post import Post
from models.profile import Profile
import models.like  # Import the like association table to ensure it's registered in metadata.

In [2]:
Base.metadata.drop_all(bind=engine)

Base.metadata.create_all(bind=engine)  # Tells SQLAlchemy: “Look at all models that inherit from Base (like User) and create their tables in the database if they don’t already exist.”
# When SQLAlchemy builds the metadata, it collects all the tables and models that were imported.

# Opens a session (like opening a conversation with the database).
# You use this session to add, read, update, and delete rows.
session = SessionLocal()

2025-09-20 22:05:56,649 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-09-20 22:05:56,653 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-20 22:05:56,656 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-09-20 22:05:56,656 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-20 22:05:56,658 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-09-20 22:05:56,658 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-20 22:05:56,663 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:56,667 INFO sqlalchemy.engine.Engine DESCRIBE `sqlalchemy_basics`.`users`
2025-09-20 22:05:56,671 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-20 22:05:56,676 INFO sqlalchemy.engine.Engine DESCRIBE `sqlalchemy_basics`.`posts`
2025-09-20 22:05:56,676 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-20 22:05:56,682 INFO sqlalchemy.engine.Engine DESCRIBE `sqlalchemy_basics`.`profiles`
2025-09-20 22:05:56,685 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-20 22:05:56,6

In [3]:
## CREATE

# add multiple users at once

user_john = User(username="john", email="john@example.com")
user_alice = User(username="alice", email="alice@example.com")
user_mark = User(username="mark", email="mark@example.com")
session.add_all([user_john, user_alice, user_mark])  # Marks it to be added.
session.commit()  # Actually saves it in the database.

# add a single user

new_user = User(username="islam", email="islam@example.com")
session.add(new_user)  # Marks it to be added.
session.commit()  # Actually saves it in the database.
session.refresh(new_user)  # Updates the Python object with fresh data from the DB (like its auto-generated id).
print("Inserted:", new_user.id, new_user.username)

users = session.query(User).all() 
for user in users:
    post = Post(title=f"Post by {user.username}", content=f"A post by {user.username}", user_id=user.id)
    session.add(post)
    session.commit()    

2025-09-20 22:05:57,292 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:57,297 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email) VALUES (%(username)s, %(email)s)
2025-09-20 22:05:57,300 INFO sqlalchemy.engine.Engine [generated in 0.00239s] {'username': 'john', 'email': 'john@example.com'}
2025-09-20 22:05:57,305 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email) VALUES (%(username)s, %(email)s)
2025-09-20 22:05:57,308 INFO sqlalchemy.engine.Engine [cached since 0.01072s ago] {'username': 'alice', 'email': 'alice@example.com'}
2025-09-20 22:05:57,313 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email) VALUES (%(username)s, %(email)s)
2025-09-20 22:05:57,318 INFO sqlalchemy.engine.Engine [cached since 0.02021s ago] {'username': 'mark', 'email': 'mark@example.com'}
2025-09-20 22:05:57,326 INFO sqlalchemy.engine.Engine COMMIT
2025-09-20 22:05:57,338 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:57,339 INFO sqlal

In [4]:
## UPDATE

user = session.query(User).filter(User.username == "alice").first()  # Finds the user with username 'alice'.
user.email = "alice@new.com"  # Change the email in Python memory.
session.commit()  # Pushes the update into the database.
session.refresh(user)  # Refreshes the object from the DB (so it’s up-to-date).
print("Updated:", user)

2025-09-20 22:05:57,508 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:57,521 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users 
WHERE users.username = %(username_1)s 
 LIMIT %(param_1)s
2025-09-20 22:05:57,530 INFO sqlalchemy.engine.Engine [generated in 0.00896s] {'username_1': 'alice', 'param_1': 1}
2025-09-20 22:05:57,540 INFO sqlalchemy.engine.Engine UPDATE users SET email=%(email)s WHERE users.id = %(users_id)s
2025-09-20 22:05:57,541 INFO sqlalchemy.engine.Engine [generated in 0.00141s] {'email': 'alice@new.com', 'users_id': 2}
2025-09-20 22:05:57,544 INFO sqlalchemy.engine.Engine COMMIT
2025-09-20 22:05:57,560 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:57,570 INFO sqlalchemy.engine.Engine SELECT users.id, users.username, users.email 
FROM users 
WHERE users.id = %(pk_1)s
2025-09-20 22:05:57,572 INFO sqlalchemy.engine.Engine [cached since 0.2124s ago] {'pk_1': 2

In [5]:
## DELETE

session.delete(user)  # Marks this row for deletion.
session.commit()  # Executes the deletion in the DB.
print("User Deleted.")

2025-09-20 22:05:57,633 INFO sqlalchemy.engine.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.content AS posts_content, posts.user_id AS posts_user_id 
FROM posts 
WHERE %(param_1)s = posts.user_id
2025-09-20 22:05:57,638 INFO sqlalchemy.engine.Engine [generated in 0.00490s] {'param_1': 2}
2025-09-20 22:05:57,650 INFO sqlalchemy.engine.Engine SELECT profiles.id AS profiles_id, profiles.bio AS profiles_bio, profiles.user_id AS profiles_user_id 
FROM profiles 
WHERE %(param_1)s = profiles.user_id
2025-09-20 22:05:57,655 INFO sqlalchemy.engine.Engine [generated in 0.00532s] {'param_1': 2}
2025-09-20 22:05:57,674 INFO sqlalchemy.engine.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.content AS posts_content, posts.user_id AS posts_user_id 
FROM posts, likes 
WHERE %(param_1)s = likes.user_id AND posts.id = likes.post_id
2025-09-20 22:05:57,676 INFO sqlalchemy.engine.Engine [generated in 0.00220s] {'param_1': 2}
2025-09-20 22:05:57,700 INFO sqlal

In [6]:
## READ all users with filtering and ordering

# Filter_by: Only works with simple equality checks (=), You pass the field name directly as a keyword argument.        
users = session.query(User).all()  # get all users
first_user = session.query(User).first()  # get the first user
last_user = session.query(User).order_by(User.id.desc()).first()  # get the last user
user_by_id = session.query(User).get(2)  # get user by primary key (id=2)
user_by_username = session.query(User).filter_by(username='alice').first()  # get user by username

# Filter: More powerful → supports all conditions (==, !=, <, >, in_, like, etc.), You must use the model.field syntax.
result = session.query(User).filter(User.username == "alice").all()
result = session.query(User).filter(User.id > 5).all()
result = session.query(User).filter(and_(User.id > 5, User.username != "bob")).all()
result = session.query(User).filter(or_(User.username == "alice", User.username == "bob")).all()
result = session.query(User).filter(User.username.like("%a%")).all()
result = session.query(User).filter(User.email.contains("@gmail.com")).all()
result = session.query(User).filter(User.username.startswith("a")).all()
result = session.query(User).filter(User.username.endswith("z")).all()

# Ordering and Pagination
result = session.query(User).order_by(User.username).all()
result = session.query(User).order_by(User.id.desc()).all()
session.query(User).limit(5).all()     # first 5 users
session.query(User).offset(10).all()  # skip first 10 users

# Count
result = session.query(User).count()  # total number of users
print(result)

# Selecting Only Some Columns
result = session.query(User.username, User.email).all()  # list of tuples (username, email)

# Selecting Distinct Values
result = session.query(User.username).distinct().all()  # unique usernames

# Combining Filters = Using AND
result = session.query(User).filter(User.username == 'islam').filter(User.id > 3).all()  # username='islam' and id>3
print(result)

# IN / NOT IN
result = session.query(User).filter(User.id.in_([1, 2, 3])).all()  # id in (1, 2, 3)
result = session.query(User).filter(~User.id.in_([1, 2, 3])).all()  # id not in (1, 2, 3)

# NULL Checks
result = session.query(User).filter(User.email == None).all()  # email is NULL
result = session.query(User).filter(User.email != None).all()  # email is NOT NULL


2025-09-20 22:05:57,917 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:57,921 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users
2025-09-20 22:05:57,924 INFO sqlalchemy.engine.Engine [cached since 0.5547s ago] {}
2025-09-20 22:05:57,936 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users 
 LIMIT %(param_1)s
2025-09-20 22:05:57,939 INFO sqlalchemy.engine.Engine [generated in 0.00265s] {'param_1': 1}
2025-09-20 22:05:57,943 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users ORDER BY users.id DESC 
 LIMIT %(param_1)s
2025-09-20 22:05:57,949 INFO sqlalchemy.engine.Engine [generated in 0.00619s] {'param_1': 1}
2025-09-20 22:05:57,956 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email

/tmp/ipykernel_125173/3059670997.py:7: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  user_by_id = session.query(User).get(2)  # get user by primary key (id=2)


In [7]:
## One-to-Many Relationship Example

new_user = User(username='ahmed', email='ahmed@gmail.com')
session.add(new_user)
session.commit()
session.refresh(new_user)


post1 = Post(title='First Post', content='This is the first post', user_id=new_user.id, author=new_user)
post2 = Post(title='Second Post', content='This is the second post', user_id=new_user.id, author=new_user)
session.add_all([post1, post2])
session.commit()

user = session.query(User).order_by(User.id.desc()).first()
print("User:", user)
print("Posts:", user.posts)  # Access related posts via the relationship attribute.

2025-09-20 22:05:59,457 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email) VALUES (%(username)s, %(email)s)
2025-09-20 22:05:59,458 INFO sqlalchemy.engine.Engine [cached since 2.161s ago] {'username': 'ahmed', 'email': 'ahmed@gmail.com'}
2025-09-20 22:05:59,464 INFO sqlalchemy.engine.Engine COMMIT
2025-09-20 22:05:59,481 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:59,485 INFO sqlalchemy.engine.Engine SELECT users.id, users.username, users.email 
FROM users 
WHERE users.id = %(pk_1)s
2025-09-20 22:05:59,486 INFO sqlalchemy.engine.Engine [cached since 2.127s ago] {'pk_1': 5}
2025-09-20 22:05:59,489 INFO sqlalchemy.engine.Engine INSERT INTO posts (title, content, user_id) VALUES (%(title)s, %(content)s, %(user_id)s)
2025-09-20 22:05:59,490 INFO sqlalchemy.engine.Engine [cached since 2.114s ago] {'title': 'First Post', 'content': 'This is the first post', 'user_id': 5}
2025-09-20 22:05:59,491 INFO sqlalchemy.engine.Engine INSERT INTO posts (title, content

In [8]:
## One-to-One Relationship Example
users = session.query(User).all()

for user in users:
    profile = Profile(bio=f"Hello, I'm {user.username}!", user_id=user.id, user=user)
    session.add(profile)
    session.commit()
    
print("profiles added")

2025-09-20 22:05:59,529 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users
2025-09-20 22:05:59,531 INFO sqlalchemy.engine.Engine [cached since 2.161s ago] {}
2025-09-20 22:05:59,535 INFO sqlalchemy.engine.Engine SELECT profiles.id AS profiles_id, profiles.bio AS profiles_bio, profiles.user_id AS profiles_user_id 
FROM profiles 
WHERE %(param_1)s = profiles.user_id
2025-09-20 22:05:59,538 INFO sqlalchemy.engine.Engine [cached since 1.888s ago] {'param_1': 1}
2025-09-20 22:05:59,541 INFO sqlalchemy.engine.Engine INSERT INTO profiles (bio, user_id) VALUES (%(bio)s, %(user_id)s)
2025-09-20 22:05:59,543 INFO sqlalchemy.engine.Engine [generated in 0.00186s] {'bio': "Hello, I'm john!", 'user_id': 1}
2025-09-20 22:05:59,548 INFO sqlalchemy.engine.Engine COMMIT
2025-09-20 22:05:59,555 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:59,556 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, us

In [9]:
## Many-to-Many Relationship Example
for user in session.query(User).all():
    for post in session.query(Post).all():
        if post.author != user:  # A user cannot like their own post
            user.liked_posts.append(post)  # User likes the post
    session.commit()

2025-09-20 22:05:59,657 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:59,658 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users
2025-09-20 22:05:59,662 INFO sqlalchemy.engine.Engine [cached since 2.292s ago] {}
2025-09-20 22:05:59,667 INFO sqlalchemy.engine.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.content AS posts_content, posts.user_id AS posts_user_id 
FROM posts
2025-09-20 22:05:59,670 INFO sqlalchemy.engine.Engine [generated in 0.00292s] {}
2025-09-20 22:05:59,673 INFO sqlalchemy.engine.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.content AS posts_content, posts.user_id AS posts_user_id 
FROM posts, likes 
WHERE %(param_1)s = likes.user_id AND posts.id = likes.post_id
2025-09-20 22:05:59,674 INFO sqlalchemy.engine.Engine [cached since 2.001s ago] {'param_1': 1}
2025-09-20 22:05:59,685 INFO sqlalchemy.engine.Engine INSERT INTO likes

In [10]:
## INNER JOIN

results = session.query(User).join(Post, User.id == Post.user_id).all()  # Join User with Post where User.id matches Post.user_id ang get only the users who have posts
for user in results:
    print(f"User: {user.username}, Posts: {[post.title for post in user.posts]}")  # get user posts from the relationship

results = session.query(User, Post).join(Post, User.id == Post.user_id).all()  # Join User with Post where User.id matches Post.user_id and get user and their posts together without using the relationship
for user, post in results:
    print(f"User: {user.username}, Post: {post.title}")

results = session.query(User.email, Post.title).join(Post, User.id == Post.user_id).all()  # Join User with Post where User.id matches Post.user_id and get user email and their post title together without using the relationship
for email, title in results:
    print(f"Email: {email}, Post Title: {title}")

stmt = select(User.username, Post.title).select_from(User).join(Post, User.id == Post.user_id)  # Using select() to join User with Post and get username and post title 
results = session.execute(stmt).all()
for username, title in results:
    print(f"Username: {username}, Post Title: {title}")

stmt = select(User).join(Post, User.id == Post.user_id).where(Post.title.like('%First%')) # Using select() to join User with Post and filter posts with title containing 'First'
results = session.execute(stmt).all()
print(f"results: {results}")

2025-09-20 22:05:59,814 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-20 22:05:59,818 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users INNER JOIN posts ON users.id = posts.user_id
2025-09-20 22:05:59,820 INFO sqlalchemy.engine.Engine [generated in 0.00203s] {}
2025-09-20 22:05:59,824 INFO sqlalchemy.engine.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.content AS posts_content, posts.user_id AS posts_user_id 
FROM posts 
WHERE %(param_1)s = posts.user_id
2025-09-20 22:05:59,825 INFO sqlalchemy.engine.Engine [cached since 2.192s ago] {'param_1': 1}
User: john, Posts: ['Post by john']
2025-09-20 22:05:59,831 INFO sqlalchemy.engine.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.content AS posts_content, posts.user_id AS posts_user_id 
FROM posts 
WHERE %(param_1)s = posts.user_id
2025-09-20 22:05:59,832 INFO sqlalchemy.engine.Engine [cached since 2.199s

In [11]:
## Left Join
results = session.query(User).outerjoin(Post, User.id == Post.user_id).all()  # Left Join User with Post where User.id matches Post.user_id and get all users even if they have no posts
for user in results:
    print(f"User: {user.username}, Posts: {[post.title for post in user.posts]}")  # get user posts from the relationship

# left join using select()
stmt = select(User).outerjoin(Post, User.id == Post.user_id)  # Left Join User with Post using select()
results = session.execute(stmt).scalars().all()
for user in results:
    print(user.username, [post.title for post in user.posts])

2025-09-20 22:05:59,873 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email 
FROM users LEFT OUTER JOIN posts ON users.id = posts.user_id
2025-09-20 22:05:59,874 INFO sqlalchemy.engine.Engine [generated in 0.00128s] {}
User: john, Posts: ['Post by john']
User: mark, Posts: ['Post by mark']
User: islam, Posts: ['Post by islam']
User: ahmed, Posts: ['First Post', 'Second Post']
2025-09-20 22:05:59,882 INFO sqlalchemy.engine.Engine SELECT users.id, users.username, users.email 
FROM users LEFT OUTER JOIN posts ON users.id = posts.user_id
2025-09-20 22:05:59,884 INFO sqlalchemy.engine.Engine [generated in 0.00213s] {}
john ['Post by john']
mark ['Post by mark']
islam ['Post by islam']
ahmed ['First Post', 'Second Post']
ahmed ['First Post', 'Second Post']


In [12]:
## Full Outer Join (if supported by the database)

# Note: Not all databases support FULL OUTER JOIN. SQLite, for example, does not.

left = select(User.username, Post.title).select_from(User).outerjoin(Post, User.id == Post.user_id)  # Left Join User with Post
right = select(User.username, Post.title).select_from(Post).outerjoin(User, User.id == Post.user_id)  # Right Join User with Post (simulated using left join from Post side)
full_outer_join = left.union(right)  # Combine both sides to simulate FULL OUTER JOIN
results = session.execute(full_outer_join).all()
for username, title in results:
    print(f"Username: {username}, Post Title: {title}")

2025-09-20 22:05:59,905 INFO sqlalchemy.engine.Engine SELECT users.username, posts.title 
FROM users LEFT OUTER JOIN posts ON users.id = posts.user_id UNION SELECT users.username, posts.title 
FROM posts LEFT OUTER JOIN users ON users.id = posts.user_id
2025-09-20 22:05:59,907 INFO sqlalchemy.engine.Engine [generated in 0.00172s] {}
Username: ahmed, Post Title: First Post
Username: ahmed, Post Title: Second Post
Username: islam, Post Title: Post by islam
Username: john, Post Title: Post by john
Username: mark, Post Title: Post by mark


In [13]:
session.close()  # Closes the session (conversation) with the database.

2025-09-20 22:05:59,934 INFO sqlalchemy.engine.Engine ROLLBACK


### Notes:

    1. session.flush() : "Send it to DB now, but I can still cancel later. using session.rollback() "
    2. session.commit() : "Save it for real, permanent in DB."
    3. session.refresh(obj) : "Reload this object from DB."

    4. You can get the ID right after flush() or commit() — you don’t need refresh() for IDs.
    5. You use refresh() only if you want the latest DB state for the object.

    6. Cascading
        - `delete-orphan`: Children deleted when detached from parent
        - `all` cascade includes all other types `save-update, merge, refresh-expire, expunge, delete`

        - One-to-many → cascade="all, delete-orphan" on parent side.
        - One-to-one → same, but only on one side.
        - Many-to-many → don’t use delete-orphan, only `all` on one side

    7. Do we need to use both cascade options with ondelete / onupdate ?

        - Best practice: Use both if you want consistent behavior in ORM and DB.
        - cascade="all, delete-orphan" → handles ORM-side deletion when working in Python.
        - ondelete="CASCADE" → ensures DB integrity even if something deletes the parent row outside of SQLAlchemy (like raw SQL, another app, or a migration).

    8. backref vs back_populates

        - back_populates → You explicitly declare the relationship on both sides, You must tell SQLAlchemy which attribute on the other model points back.
        - backref → Shortcut: define the relationship on only one side, SQLAlchemy will auto-generate the reverse relationship for you.